# 해커톤 프로젝트 평가 시스템 - 분석 노트북

이 노트북은 해커톤 프로젝트 평가 시스템의 핵심 기능을 단계별로 실행하고 결과를 확인할 수 있도록 구성되었습니다.

## 개요

해커톤 프로젝트 평가 시스템은 다음과 같은 주요 기능을 제공합니다:

1. **멀티모달 데이터 분석**: 비디오, 문서, 프레젠테이션 등 다양한 형식의 프로젝트 자료 분석
2. **프로젝트 유형 분류**: PainKiller/Vitamin 분류 모델을 통한 프로젝트 특성 파악
3. **다양한 평가 체인**: 9가지 평가 영역(비즈니스 가치, 기술적 실현가능성 등)을 통한 종합 평가
4. **병렬 처리 기능**: ChainExecutor를 통한 효율적인 평가 처리

이 노트북에서는 각 단계별 실행 과정과 결과를 시각적으로 확인할 수 있습니다.

## 1. 필요 라이브러리 및 모듈 임포트

In [ ]:
# 필요한 라이브러리 임포트
import os
import sys
from pathlib import Path
import json
import time
from datetime import datetime
from pprint import pprint

# 데이터 분석 및 시각화 라이브러리
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.graph_objects as go
import plotly.express as px
from IPython.display import display, HTML

# 병렬 처리 라이브러리
from langchain_core.runnables import RunnableParallel
from concurrent.futures import ThreadPoolExecutor

# 시스템 관련 모듈 임포트
from src.analysis import (
    VideoAnalysis,
    DocumentAnalysis,
    PresentationAnalysis
)
from src.chain import (
    AccessibilityChain,
    BusinessValueChain,
    CostAnalysisChain,
    InnovationChain,
    NetworkEffectChain,
    SocialImpactChain,
    SustainabilityChain,
    TechnicalFeasibilityChain,
    UserEngagementChain,
)
from src.chain.chain_executor import ChainExecutor
from src.classifier import ProjectTypeClassifier
from src.config.weight_manager import WeightManager

# 경로 확인
print(f"현재 작업 디렉토리: {os.getcwd()}")

## 2. 데이터 입력 설정

In [ ]:
# 입력 데이터 설정 (S3 URI 또는 로컬 파일 경로)
# 실제 환경에서는 환경 변수나 구성 파일에서 로드

# 테스트용 기본 데이터
default_video_uri = "s3://hackathon-test/video/sample_pitch.mp4"
default_document_uri = "s3://hackathon-test/docs/project_description.txt"
default_presentation_uri = "s3://hackathon-test/presentation/slides.pdf"

# 경로 설정을 위한 위젯 (주피터 노트북 실행 환경에서 활성화됨)
from ipywidgets import interact, widgets

video_input = widgets.Text(
    value=default_video_uri,
    placeholder='비디오 파일 S3 URI 또는 로컬 경로',
    description='비디오:',
    disabled=False
)

document_input = widgets.Text(
    value=default_document_uri,
    placeholder='문서 파일 S3 URI 또는 로컬 경로',
    description='문서:',
    disabled=False
)

presentation_input = widgets.Text(
    value=default_presentation_uri,
    placeholder='발표자료 S3 URI 또는 로컬 경로',
    description='발표자료:',
    disabled=False
)

# 위젯 표시
display(video_input)
display(document_input)
display(presentation_input)

# 입력 데이터 구성
def get_input_data():
    return {
        "video_uri": video_input.value,
        "document_uri": document_input.value,
        "presentation_uri": presentation_input.value
    }

# 현재 입력 확인
user_input = get_input_data()
print("\n설정된 입력 데이터:")
pprint(user_input)

## 3. 데이터 분석 실행

In [ ]:
# 분석 클래스 인스턴스 생성
video_analysis = VideoAnalysis()
presentation_analysis = PresentationAnalysis()
document_analysis = DocumentAnalysis()

print("분석 클래스 초기화 완료")

In [ ]:
# 비디오 분석 실행
print("비디오 분석 시작...")
try:
    video_analysis_result = video_analysis.invoke({"s3_uri": user_input["video_uri"]})
    print("비디오 분석 완료")
except Exception as e:
    print(f"비디오 분석 중 오류 발생: {e}")
    video_analysis_result = {
        "error": str(e),
        "content": "분석 실패",
        "summary": "오류로 인해 분석할 수 없습니다.",
        "keywords": []
    }

# 비디오 분석 결과 요약 출력
print("\n비디오 분석 결과 요약:")
if "error" not in video_analysis_result:
    print(f"- 키워드: {', '.join(video_analysis_result.get('keywords', ['없음'])[:5])}")
    print(f"- 요약: {video_analysis_result.get('summary', '요약 없음')[:100]}...")
else:
    print(f"- 오류: {video_analysis_result.get('error')}")

In [ ]:
# 문서 분석 실행
print("문서 분석 시작...")
try:
    document_analysis_result = document_analysis.invoke({"s3_uri": user_input["document_uri"]})
    print("문서 분석 완료")
except Exception as e:
    print(f"문서 분석 중 오류 발생: {e}")
    document_analysis_result = {
        "error": str(e),
        "content": "분석 실패",
        "summary": "오류로 인해 분석할 수 없습니다.",
        "keywords": []
    }

# 문서 분석 결과 요약 출력
print("\n문서 분석 결과 요약:")
if "error" not in document_analysis_result:
    print(f"- 키워드: {', '.join(document_analysis_result.get('keywords', ['없음'])[:5])}")
    print(f"- 요약: {document_analysis_result.get('summary', '요약 없음')[:100]}...")
else:
    print(f"- 오류: {document_analysis_result.get('error')}")

In [ ]:
# 발표자료 분석 실행
print("발표자료 분석 시작...")
try:
    presentation_analysis_result = presentation_analysis.invoke({"s3_uri": user_input["presentation_uri"]})
    print("발표자료 분석 완료")
except Exception as e:
    print(f"발표자료 분석 중 오류 발생: {e}")
    presentation_analysis_result = {
        "error": str(e),
        "content": "분석 실패",
        "summary": "오류로 인해 분석할 수 없습니다.",
        "keywords": []
    }

# 발표자료 분석 결과 요약 출력
print("\n발표자료 분석 결과 요약:")
if "error" not in presentation_analysis_result:
    print(f"- 키워드: {', '.join(presentation_analysis_result.get('keywords', ['없음'])[:5])}")
    print(f"- 요약: {presentation_analysis_result.get('summary', '요약 없음')[:100]}...")
else:
    print(f"- 오류: {presentation_analysis_result.get('error')}")

### 분석 결과 시각화

In [ ]:
# 키워드 빈도수 시각화
all_keywords = []
all_keywords.extend(video_analysis_result.get('keywords', []))
all_keywords.extend(document_analysis_result.get('keywords', []))
all_keywords.extend(presentation_analysis_result.get('keywords', []))

from collections import Counter
keyword_counts = Counter(all_keywords)

# 상위 10개 키워드 추출
top_keywords = dict(keyword_counts.most_common(10))

# 막대 그래프로 표시
plt.figure(figsize=(10, 6))
sns.barplot(x=list(top_keywords.keys()), y=list(top_keywords.values()))
plt.title('상위 10개 키워드 빈도수')
plt.xlabel('키워드')
plt.ylabel('빈도수')
plt.xticks(rotation=45)
plt.tight_layout()
plt.show()

In [ ]:
# 분석 결과 종합
evaluator_chain_input = {
    "video_analysis": video_analysis_result,
    "document_analysis": document_analysis_result,
    "presentation_analysis": presentation_analysis_result
}

print("평가 체인 입력 데이터 준비 완료")

## 4. 프로젝트 유형 분류

In [ ]:
# 프로젝트 유형 분류기 초기화
print("프로젝트 유형 분류기 초기화...")
classifier = ProjectTypeClassifier()
print("분류기 초기화 완료")

In [ ]:
# 분류 실행
print("프로젝트 유형 분류 실행...")
try:
    type_result = classifier.classify(evaluator_chain_input)
    print("분류 완료")
except Exception as e:
    print(f"분류 중 오류 발생: {e}")
    type_result = {
        "project_type": "balanced",
        "confidence": 0.0,
        "painkiller_score": 0.5,
        "vitamin_score": 0.5,
        "reasoning": f"분류 오류: {str(e)}",
        "warning_message": f"오류로 인해 기본값으로 처리되었습니다: {str(e)}"
    }

# 분류 결과 출력
print("\n=== 프로젝트 유형 분류 결과 ===")
print(f"프로젝트 유형: {type_result['project_type']}")
print(f"신뢰도: {type_result['confidence']:.3f}")
print(f"PainKiller 점수: {type_result['painkiller_score']:.3f}")
print(f"Vitamin 점수: {type_result['vitamin_score']:.3f}")
print(f"분류 근거: {type_result['reasoning'][:200]}...")
if type_result.get('warning_message'):
    print(f"경고: {type_result['warning_message']}")
print()

In [ ]:
# 분류 결과 시각화
fig = go.Figure()

# PainKiller vs Vitamin 점수 비교 막대 그래프
fig.add_trace(go.Bar(
    x=['PainKiller', 'Vitamin'],
    y=[type_result['painkiller_score'], type_result['vitamin_score']],
    text=[f"{type_result['painkiller_score']:.2f}", f"{type_result['vitamin_score']:.2f}"],
    textposition='auto',
    marker_color=['#FF5733', '#33FF57'],
    name='점수'
))

fig.update_layout(
    title='PainKiller vs Vitamin 점수 비교',
    xaxis_title='프로젝트 유형',
    yaxis_title='점수 (0-1)',
    yaxis=dict(range=[0, 1])
)

fig.show()

# 신뢰도 게이지 차트
fig = go.Figure(go.Indicator(
    mode="gauge+number",
    value=type_result['confidence'] * 100,
    domain={'x': [0, 1], 'y': [0, 1]},
    title={'text': "분류 신뢰도"},
    gauge={
        'axis': {'range': [None, 100]},
        'bar': {'color': "#1F77B4"},
        'steps': [
            {'range': [0, 30], 'color': "#FFDDDD"},
            {'range': [30, 70], 'color': "#FFEEAA"},
            {'range': [70, 100], 'color': "#DDFFDD"}
        ],
    }
))

fig.update_layout(
    title='분류 신뢰도 점수'
)

fig.show()

In [ ]:
# 평가 체인 입력에 분류 결과 추가
evaluator_chain_input["project_type"] = type_result
print("평가 체인 입력 데이터에 분류 결과 추가 완료")

## 5. 평가 체인 실행

In [ ]:
# 모든 평가 체인 인스턴스 생성
print("평가 체인 초기화...")
evaluation_chains = {
    "business_value": BusinessValueChain(),
    "accessibility": AccessibilityChain(),
    "innovation": InnovationChain(),
    "cost_analysis": CostAnalysisChain(),
    "network_effect": NetworkEffectChain(),
    "social_impact": SocialImpactChain(),
    "sustainability": SustainabilityChain(),
    "technical_feasibility": TechnicalFeasibilityChain(),
    "user_engagement": UserEngagementChain(),
}
print("평가 체인 초기화 완료")

In [ ]:
# 체인 실행기 초기화
print("체인 실행기 초기화...")
executor = ChainExecutor()

# 진행 상황 콜백 함수 정의
def progress_callback(chain_name, current, total):
    print(f"진행 상황: {chain_name} ({current+1}/{total})")

# 콜백 설정
executor.set_progress_callback(progress_callback)
print("체인 실행기 초기화 및 콜백 설정 완료")

In [ ]:
# 모든 평가 체인 병렬 실행
print("평가 체인 실행 시작...")
start_time = time.time()

execution_result = executor.execute_all(evaluator_chain_input)

end_time = time.time()
execution_time = end_time - start_time
print(f"평가 체인 실행 완료 (소요 시간: {execution_time:.2f}초)")

In [ ]:
# 실행 결과 요약
results = execution_result["results"]
error_count = execution_result["metadata"]["error_count"]

print(f"총 평가 체인: {len(results)}개, 오류 발생 체인: {error_count}개")

# 오류 발생 체인 확인
if error_count > 0:
    print("\n오류 발생 체인:")
    for chain_name, result in results.items():
        if "error" in result:
            print(f"- {chain_name}: {result['error']}")

# 점수 추출
scores = executor.get_scores(results)

print("\n각 평가 항목별 점수:")
for category, score in scores.items():
    print(f"- {category}: {score:.2f}/10")

## 6. 결과 분석 및 시각화

In [ ]:
# 평가 결과 데이터프레임 생성
scores_df = pd.DataFrame(list(scores.items()), columns=['평가항목', '점수'])

# 점수 기준 내림차순 정렬
scores_df = scores_df.sort_values('점수', ascending=False)

# 평가 항목 한글명 매핑
category_names = {
    'business_value': '비즈니스 가치',
    'technical_feasibility': '기술적 실현가능성',
    'innovation': '혁신성',
    'user_engagement': '사용자 참여도',
    'accessibility': '접근성',
    'social_impact': '사회적 영향',
    'sustainability': '지속가능성',
    'network_effect': '네트워크 효과',
    'cost_analysis': '비용 분석'
}

scores_df['평가항목_한글'] = scores_df['평가항목'].map(lambda x: category_names.get(x, x))

# 데이터프레임 출력
scores_df[['평가항목_한글', '점수']]

In [ ]:
# 막대 차트로 점수 비교
plt.figure(figsize=(12, 6))
ax = sns.barplot(x='평가항목_한글', y='점수', data=scores_df)
plt.title('평가 항목별 점수')
plt.xlabel('평가 항목')
plt.ylabel('점수 (0-10)')
plt.ylim(0, 10)
plt.xticks(rotation=45)
plt.tight_layout()

# 점수 표시
for i, v in enumerate(scores_df['점수']):
    ax.text(i, v + 0.1, f"{v:.2f}", ha='center')

plt.show()

In [ ]:
# 레이더 차트 (Plotly)
categories = scores_df['평가항목_한글'].tolist()
values = scores_df['점수'].tolist()

# 첫번째 카테고리를 마지막에 추가하여 차트를 닫음
categories.append(categories[0])
values.append(values[0])

fig = go.Figure()

fig.add_trace(go.Scatterpolar(
    r=values,
    theta=categories,
    fill='toself',
    name='점수',
    line_color='rgb(31, 119, 180)',
    fillcolor='rgba(31, 119, 180, 0.3)'
))

fig.update_layout(
    title='평가 항목별 점수 레이더 차트',
    polar=dict(
        radialaxis=dict(
            visible=True,
            range=[0, 10]
        )
    )
)

fig.show()

In [ ]:
# 히트맵 생성 - 평가 결과 간의 상관관계

# 평가 결과에서 점수 추출
def extract_scores_from_chain_results(results):
    score_details = {}
    
    for chain_name, result in results.items():
        if isinstance(result, dict) and "sub_scores" in result:
            # 세부 점수가 있는 경우
            for sub_category, sub_score in result["sub_scores"].items():
                score_details[f"{chain_name}_{sub_category}"] = sub_score
    
    return score_details

# 세부 점수 추출
sub_scores = extract_scores_from_chain_results(results)

# 세부 점수가 있으면 히트맵 생성
if sub_scores:
    sub_scores_df = pd.DataFrame([sub_scores])
    corr = sub_scores_df.T.corr()
    
    plt.figure(figsize=(10, 8))
    sns.heatmap(corr, annot=True, cmap='coolwarm', vmin=-1, vmax=1)
    plt.title('평가 세부 항목 간 상관관계')
    plt.tight_layout()
    plt.show()
else:
    print("세부 평가 점수가 없어 히트맵을 생성할 수 없습니다.")